# Demostración Integral: Detección de Regiones Brillantes (Lámparas)

## Objetivo
Implementar y comparar múltiples algoritmos de visión computacional para la detección automática de lámparas en imágenes, demostrando la efectividad de diferentes enfoques técnicos.

## Algoritmos Implementados
1. **Pipeline Principal**: Detección integral con umbral adaptativo
2. **Comparación de Umbrales**: Fijo vs Adaptativo vs Otsu
3. **Filtros de Pre-procesamiento**: Gaussian Blur y reducción de ruido
4. **Operaciones Morfológicas**: Apertura, cierre y refinamiento
5. **Segmentación Avanzada**: K-means y crecimiento de regiones
6. **Análisis Cuantitativo**: Métricas de área, perímetro y centroide

In [8]:
# Importar librerías necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from sklearn.cluster import KMeans
import os

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

# Lista de imágenes a procesar
imagenes = ['../image/1.png', '../image/2.png', '../image/3.png', '../image/4.png']

# Crear directorio de resultados
os.makedirs('../image/resultados/demostracion/', exist_ok=True)

print("✅ Librerías cargadas correctamente")
print(f"📁 Procesaremos {len(imagenes)} imágenes")

✅ Librerías cargadas correctamente
📁 Procesaremos 4 imágenes


## 1. ALGORITMO PRINCIPAL: Pipeline Integral de Detección

Este es nuestro algoritmo estrella que combina múltiples técnicas optimizadas específicamente para la detección de lámparas.

In [ ]:
def detectar_lamparas_pipeline_principal(imagen_path, mostrar_pasos=True):
    """
    Pipeline principal optimizado para detección de lámparas
    """
    # 1. Cargar imagen
    img_color = cv2.imread(imagen_path)
    if img_color is None:
        return None, None
    
    # 2. Convertir a escala de grises
    gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    # 3. Suavizado gaussiano (reducir ruido)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 4. Umbral adaptativo (optimizado para regiones brillantes)
    binary = cv2.adaptiveThreshold(blurred, 255,
                                   cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY,
                                   35,  # tamaño del bloque
                                   -10) # constante de ajuste
    
    # 5. Operaciones morfológicas (refinamiento)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)  # Eliminar ruido
    morph = cv2.morphologyEx(morph, cv2.MORPH_CLOSE, kernel)  # Rellenar huecos
    
    # 6. Análisis de regiones
    labeled = label(morph)
    props = regionprops(labeled)
    
    # 7. Filtrar y visualizar resultados
    result = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    lamparas_detectadas = []
    
    for i, region in enumerate(props, 1):
        if region.area >= 100:  # Filtrar objetos pequeños
            y0, x0 = region.centroid
            minr, minc, maxr, maxc = region.bbox
            
            # Dibujar bounding box y centroide
            cv2.rectangle(result, (minc, minr), (maxc, maxr), (0, 0, 255), 2)
            cv2.circle(result, (int(x0), int(y0)), 5, (0, 255, 0), -1)
            cv2.putText(result, f'L{i}', (minc, minr-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            lamparas_detectadas.append({
                'id': i,
                'area': region.area,
                'perimetro': region.perimeter,
                'centroide': (x0, y0)
            })
    
    if mostrar_pasos:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        axes[0,0].imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
        axes[0,0].set_title('1. Imagen Original')
        axes[0,0].axis('off')
        
        axes[0,1].imshow(gray, cmap='gray')
        axes[0,1].set_title('2. Escala de Grises')
        axes[0,1].axis('off')
        
        axes[0,2].imshow(blurred, cmap='gray')
        axes[0,2].set_title('3. Suavizado Gaussiano')
        axes[0,2].axis('off')
        
        axes[1,0].imshow(binary, cmap='gray')
        axes[1,0].set_title('4. Umbral Adaptativo')
        axes[1,0].axis('off')
        
        axes[1,1].imshow(morph, cmap='gray')
        axes[1,1].set_title('5. Morfología (Open+Close)')
        axes[1,1].axis('off')
        
        axes[1,2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        axes[1,2].set_title(f'6. Detección Final ({len(lamparas_detectadas)} lámparas)')
        axes[1,2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return result, lamparas_detectadas

# Ejecutar pipeline principal en todas las imágenes
print("🚀 EJECUTANDO PIPELINE PRINCIPAL DE DETECCIÓN\n")

resultados_principales = []
for i, path in enumerate(imagenes):
    print(f"📸 Procesando imagen {i+1}: {path}")
    resultado, lamparas = detectar_lamparas_pipeline_principal(path)
    
    if resultado is not None:
        resultados_principales.append((resultado, lamparas))
        
        # Guardar resultado
        cv2.imwrite(f'../image/resultados/demostracion/pipeline_principal_{i+1}.png', resultado)
        
        # Mostrar estadísticas
        print(f"✅ Detectadas {len(lamparas)} lámparas:")
        for lampara in lamparas[:5]:  # Mostrar solo las primeras 5
            print(f"   L{lampara['id']}: Área={lampara['area']:.0f}, "
                  f"Perímetro={lampara['perimetro']:.1f}, "
                  f"Centro=({lampara['centroide'][0]:.1f}, {lampara['centroide'][1]:.1f})")
        if len(lamparas) > 5:
            print(f"   ... y {len(lamparas)-5} lámparas más")
        print()

print(f"🎯 RESUMEN: Pipeline principal completado para {len(resultados_principales)} imágenes")

ImportError: cannot import name 'label' from 'skimage' (/home/yep/Proyectos/parcial1_visualizacion/.venv/lib64/python3.13/site-packages/skimage/__init__.py)